# Amazon, modelo supervisado

## Importar

In [3]:
# %load basic
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',200)
pd.options.display.float_format = '{:.2f}'.format
liga = '/Users/efraflores/Desktop/EF/diplo/AZ/data/'
os.listdir(liga)

['sentim.csv', 'df_cluster.csv', 'df.csv']

In [4]:
df = pd.read_csv(liga + 'df_cluster.csv', low_memory = False)
df = df.drop(columns = ['vote','image']).dropna().reset_index(drop = True)
df = df[(df['Anio'] >= 2006) & (df['Anio'] <= 2016)].copy()
backup = df.copy()
df.sample()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,category,Anio,Mes,Sem,DiaSem,reviewText_limpio,reviewText_long,reviewText_n_words,reviewText_relevant,neg,neu,pos,compound,df_cluster
718876,5.00,True,2016-10-09,A2Y3000N340K55,B00AA6CS86,Virginia,I love this thing. It's cute and I can find it...,Great deal for price. Works excellent!,0 days,No_especifica,cell_phones_and_accessories,2016,oct,40,dom,love thing cute find easily purse charge phone...,135.00,27.00,0.62,0.00,0.34,0.66,0.95,E


In [5]:
df.describe([i/10 for i in range(10)] + [.05,.95])

,overall,Anio,Sem,reviewText_long,reviewText_n_words,reviewText_relevant,neg,neu,pos,compound
count,629378.00,629378.00,629378.00,629378.00,629378.00,629378.00,629378.00,629378.00,629378.00,629378.00
mean,4.22,2012.81,26.14,536.33,97.61,0.65,0.07,0.55,0.38,0.65
std,1.14,2.37,15.59,949.01,168.50,0.12,0.10,0.17,0.19,0.46
min,1.00,2006.00,1.00,5.00,2.00,0.04,0.00,0.00,0.00,-1.00
0%,1.00,2006.00,1.00,5.00,2.00,0.04,0.00,0.00,0.00,-1.00
5%,1.00,2008.00,3.00,21.00,4.00,0.50,0.00,0.29,0.00,-0.48
10%,2.00,2009.00,5.00,32.00,6.00,0.53,0.00,0.34,0.15,0.00
20%,3.00,2011.00,10.00,91.00,17.00,0.56,0.00,0.41,0.22,0.44
30%,4.00,2012.00,15.00,135.00,25.00,0.58,0.00,0.46,0.27,0.62
40%,4.00,2013.00,20.00,169.00,32.00,0.60,0.00,0.51,0.32,0.77


## Funciones

In [6]:
import time
import math

start = time.time()

def time_exp(x):
    print(str(int(math.floor(x/60))
             ) + " minutos con " + '{:.2f}'.format(60*(x/60 - math.floor(x/60))
                                                  ) + " segundos")

In [7]:
from IPython.lib.display import Audio
import numpy as np

#Tono para cuando termina el script
def tono(a = 1000, b = 700, play_time_seconds = 1, framerate = 4410,):
    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)*np.pi
    return Audio(np.sin(a*t) + np.sin(b*t), rate = framerate, autoplay=True)

In [8]:
def outlier(df, x , p = 0.5):
    var = df[x]
    q1 = var.quantile(p/2)
    q3 = var.quantile(1 - p/2)
    iqr = q3 - q1
    df = df[(var.isnull()) | ((var >= q1 - 1.5*iqr) & (var <= q3 + 1.5*iqr))].copy()
    return df

In [9]:
def multishift(df, 
               fecha = 'reviewTime',
               target = 'neg',
               target_value = '>0',
               func = 'median',
               n = 15):
    
    #DF con todas y cada una de las fechas posibles según las fechas del DF original
    tot_dates = pd.DataFrame(pd.date_range(start = df[fecha].min(), 
                                           end = df[fecha].max()).date, 
                             columns = [fecha]).astype(str)
    
    #Count condicional de la variable target
    aux = df.pivot_table(index = fecha,
                         values = target, 
                         aggfunc = lambda x: (eval('x' + target_value)).sum() / x.count()
                        ).rename(columns = {target:'target'})
    
    #Pivot del resto de las variables numéricas con la función indicada
    df = df.pivot_table(index = fecha, aggfunc = func).join(aux.shift(-1)) #one day ahead
    
    #Se une con el total de fechas para no omitir ningún día
    df = tot_dates.merge(df, on = fecha, how = 'left').fillna(0)
    
    #Ciclo para agregar el valor de la target original (no el count) hasta n días atrás
    for i in range(1,n+1):
        df = df.join(df[[target]].shift(i).rename(columns = {target:f'{target}_{i}'}))
        
    #No es necesario contar con los primeros n registros dado el ciclo de shifts
    return df.iloc[n:,:]

In [10]:
def tad_multicat(df,cat_col):
    total = pd.DataFrame()
    for cat in set(df[cat_col]):
        aux = multishift(df[df[cat_col] == cat])
        aux['category'] = cat
        total = total.append(aux, ignore_index = True)
    return total

In [11]:
def fechas(df, fecha, formato = "%Y-%m-%d"):
    df = df.copy()
    df[fecha] = pd.to_datetime(df[fecha], format = formato)
    df['Anio'] = df[fecha].dt.year
    meses = ['ene','feb','mar','abr','may','jun',
             'jul','ago','sep','oct','nov','dic']
    meses = dict(zip(range(1,13),meses))
    #Mes en formato MMM (español)
    df['Mes'] = df[fecha].dt.month.replace(meses)
    df['Sem'] = df[fecha].dt.isocalendar().week
    diasem = ['lun','mar','mie','jue','vie','sab','dom']
    diasem = dict(zip(range(7),diasem))
    #Día de la semana en formato DDD (español)
    df['DiaSem'] = df[fecha].dt.dayofweek.replace(diasem)
    return df

## TAD

In [12]:
#df = fechas(tad_multicat(df,'category'), fecha = 'reviewTime')
df = fechas(multishift(df), fecha = 'reviewTime')
df.sample(4)

,reviewTime,Anio,Sem,compound,neg,neu,overall,pos,reviewText_long,reviewText_n_words,reviewText_relevant,verified,target,neg_1,neg_2,neg_3,neg_4,neg_5,neg_6,neg_7,neg_8,neg_9,neg_10,neg_11,neg_12,neg_13,neg_14,neg_15,Mes,DiaSem
3031,2014-04-20,2014,16,0.90,0.04,0.53,5.00,0.40,213.50,41.00,0.61,1.00,0.54,0.04,0.05,0.04,0.03,0.03,0.04,0.04,0.04,0.04,0.00,0.00,0.03,0.05,0.04,0.04,abr,dom
3131,2014-07-29,2014,31,0.66,0.00,0.51,5.00,0.43,114.00,21.00,0.68,1.00,0.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,jul,mar
1301,2009-07-25,2009,30,0.95,0.14,0.58,5.00,0.27,952.00,177.00,0.60,0.00,0.82,0.10,0.12,0.11,0.11,0.09,0.11,0.14,0.09,0.10,0.07,0.10,0.11,0.11,0.14,0.10,jul,sab
3114,2014-07-12,2014,28,0.71,0.00,0.51,5.00,0.42,118.00,22.50,0.65,1.00,0.38,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,jul,sab


## Modelo

### f(X) = y

In [13]:
df.drop(columns = 'reviewTime', inplace = True)

num = [x for x in df.describe().columns if x not in ['target']]
cat = [x for x in df.columns if x not in num + ['target']]
df = df[cat+num+['target']].copy()

X = df[cat + num].copy()
y = df['target']

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size = 0.77, 
                                                    random_state = 22)

### Preprocesamiento

In [15]:
#Encoder para categóricas
from category_encoders.cat_boost import CatBoostEncoder
encod = CatBoostEncoder()

#Escala para numéricas
from sklearn.preprocessing import MinMaxScaler
mm_x = MinMaxScaler()

#Se aplicará transformación para cada tipo de columnas
from sklearn.compose import ColumnTransformer
prep = ColumnTransformer(transformers=[('Cat_encod', encod, cat),
                                       ('Scale', mm_x, num)])

### Ensamble

In [20]:
forest.get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [25]:
scoring = 'r2'

from sklearn.ensemble import RandomForestRegressor 
forest = RandomForestRegressor()

param_forest = {'n_estimators': [x for x in range(1400, 1500, 10)],
                'max_features': ['auto', 'sqrt', 'log2'],
                'criterion': ['mse','mae'],
                'min_samples_split': [x for x in range(10, 22)],
                'min_samples_leaf': [x/1000 for x in range(1, 6)]
               }

from sklearn.model_selection import RandomizedSearchCV
search_forest = RandomizedSearchCV(param_distributions = param_forest, 
                                   cv = 4, 
                                   n_jobs = -1, 
                                   scoring = scoring,
                                   estimator = forest,
                                   verbose = 5,
                                   n_iter = 10,
                                   random_state = 22)

In [26]:
from xgboost.sklearn import XGBRegressor
xgb = XGBRegressor()

param_xgb = {'learning_rate':[x/100 for x in range(1,111)],
             'n_estimators':[x for x in range(1,111)],
             'max_depth':[x for x in range(1,11)], 
             'min_child_weight':[x for x in range(1,111)],
             'objective':['count:poisson','multi:softmax'],
             'subsample':[x/100 for x in range(50,111)], 
             'colsample_bytree':[x/100 for x in range(50,111)], 
            }

from sklearn.model_selection import RandomizedSearchCV
search_xgb = RandomizedSearchCV(param_distributions = param_xgb, 
                                cv = 4, 
                                n_jobs = -1, 
                                scoring = scoring, 
                                estimator = xgb, 
                                verbose = 5,
                                n_iter = 10,
                                random_state = 22)

In [27]:
from sklearn.ensemble import VotingRegressor
vr = VotingRegressor(estimators = [('LogReg', search_forest), 
                                    ('XGB', search_xgb)])

In [28]:
from sklearn.pipeline import Pipeline
modelo = Pipeline(steps=[('preproc', prep),
                         ('modelo', vr)])

modelo.fit(X_train,y_train).score(X_test,y_test)

Fitting 4 folds for each of 10 candidates, totalling 40 fits


KeyboardInterrupt: 

### Validación

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
cm = pd.DataFrame(confusion_matrix(y_true = y_test,
                                   y_pred = modelo.predict(X_test)))

## Tiene buena acertividad (suma de diagonal en la matriz de confusión)
display('Accuracy de {:.2%}'.format(np.asarray(cm).trace()/len(y_test)))

fig, ax = plt.subplots(figsize=(7,7)) 
sns.heatmap(pd.DataFrame([cm[col]/cm[col].sum() for col in cm.columns]), 
            annot = True,
            fmt = '.0%',
            cmap = 'Blues',
            linewidths = 0.5, 
            ax = ax)

In [ ]:
pd.DataFrame(zip(df.columns, modelo[1].estimators_[1].best_estimator_.feature_importances_),
             columns = ['Variable',
                        'Importancia']).sort_values(by = 'Importancia', ascending = False
                                                   ).head(10).reset_index(drop = True)

### Conservación

In [ ]:
import pickle
with open('AZ_modelo_sup_cont_v1.pkl', "wb") as f:
    pickle.dump(modelo, f)

import pickle    
with open('AZ_modelo_sup_cont_v1.pkl', "rb") as f:
    modelo = pickle.load(f)

## Fin

In [ ]:
#Tiempo total para correr el notebook
end = time.time()
time_exp(end - start)
tono()